In [7]:
!pip install ccxt

In [2]:
pip install vectorbt

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas_ta

Note: you may need to restart the kernel to use updated packages.


In [1]:
import ccxt
import numpy as np
import time 
from datetime import datetime
import schedule
import pandas as pd
import pandas_ta as ta

In [2]:
binance =ccxt.binance({
    'apiKey':'ciDxQ2cBaeFyCpElYqin7zEWzvRukTK70crkyDUxeouhkFkHjettIGfWw7C47PkK',
    'secret':'C4DyuZQSG3lC9mMiAEQTPIywTrD9pSi1SKcN7DuASnuWA3N3MMdKadbPrDzr2SA1'
})

In [3]:
get_market = binance.load_markets()

In [4]:
binance.markets['BTC/USDT']

{'id': 'BTCUSDT',
 'symbol': 'BTC/USDT',
 'base': 'BTC',
 'quote': 'USDT',
 'baseId': 'BTC',
 'quoteId': 'USDT',
 'active': True,
 'type': 'spot',
 'linear': None,
 'inverse': None,
 'spot': True,
 'swap': False,
 'future': False,
 'option': False,
 'margin': True,
 'contract': False,
 'contractSize': None,
 'expiry': None,
 'expiryDatetime': None,
 'optionType': None,
 'strike': None,
 'settle': None,
 'settleId': None,
 'precision': {'amount': 5, 'price': 2, 'base': 8, 'quote': 8},
 'limits': {'amount': {'min': 1e-05, 'max': 9000.0},
  'price': {'min': 0.01, 'max': 1000000.0},
  'cost': {'min': 10.0, 'max': None},
  'leverage': {'min': None, 'max': None},
  'market': {'min': 0.0, 'max': 328.06263694}},
 'info': {'symbol': 'BTCUSDT',
  'status': 'TRADING',
  'baseAsset': 'BTC',
  'baseAssetPrecision': '8',
  'quoteAsset': 'USDT',
  'quotePrecision': '8',
  'quoteAssetPrecision': '8',
  'baseCommissionPrecision': '8',
  'quoteCommissionPrecision': '8',
  'orderTypes': ['LIMIT',
   'LIM

In [5]:
from datetime import datetime
startDate = "2022-08-01"
startDate = datetime.strptime(startDate, "%Y-%m-%d")
startDate = datetime.timestamp(startDate)
startDate = int(startDate) * 1000
print(startDate)  #output: 1589846400000

1659286800000


In [6]:
bars = binance.fetch_ohlcv('BTC/USDT',timeframe='5m',limit = None,since=startDate)

In [7]:
df = pd.DataFrame(bars,columns=['time','open','high','low','close','volume'])
df

,time,open,high,low,close,volume
0,1659286800000,23768.30,23777.87,23742.00,23751.20,264.36684
1,1659287100000,23751.20,23787.32,23732.98,23761.29,430.02050
2,1659287400000,23762.28,23777.51,23746.48,23754.32,203.32356
3,1659287700000,23754.32,23765.87,23743.76,23751.69,273.45352
4,1659288000000,23752.87,23763.96,23723.52,23742.98,305.43913
...,...,...,...,...,...,...
495,1659435300000,22810.13,22814.92,22758.26,22771.86,467.44205
496,1659435600000,22771.84,22806.99,22768.16,22794.06,357.51439
497,1659435900000,22793.37,22801.92,22751.52,22769.93,391.94326
498,1659436200000,22770.97,22800.00,22758.40,22784.52,466.29267


In [8]:
df['time'] = pd.to_datetime(df['time'],unit='ms')
df = df.set_index('time')

In [9]:
df.head()

,open,high,low,close,volume
time,,,,,
2022-07-31 17:00:00,23768.30,23777.87,23742.00,23751.20,264.36684
2022-07-31 17:05:00,23751.20,23787.32,23732.98,23761.29,430.02050
2022-07-31 17:10:00,23762.28,23777.51,23746.48,23754.32,203.32356
2022-07-31 17:15:00,23754.32,23765.87,23743.76,23751.69,273.45352
2022-07-31 17:20:00,23752.87,23763.96,23723.52,23742.98,305.43913


In [10]:
def strategy(df):
    df['ema5'] = ta.ema(df['close'], length=5)
    df['ema15'] = ta.sma(df['close'], length=15)
    df['signal'] = df['ema5'] > df['ema15']

In [11]:
def get_realtime(coin='BTC/USDT', tf='1m', initial_bar=25, fetch_new=True):
    global bars, df, new_df
    
    bars = binance.fetch_ohlcv(coin, timeframe=tf, limit=initial_bar)
    df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df[:25]
    strategy(df)
    
#     fetch_new = True
    
    if fetch_new == True:
        bars2 = binance.fetch_ohlcv(coin, timeframe=tf, limit=1)        
        
        data = pd.DataFrame(bars2,columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
        new_df = pd.concat([df, data])
        
        new_df['ema5'].iloc[-1] = ta.ema(new_df['close'][-5:], length=5).iloc[-1]
        new_df['ema15'].iloc[-1] = ta.ema(new_df['close'][-15:], length=15).iloc[-1]
        new_df['signal'].iloc[-1] = new_df['ema5'].iloc[-1] > new_df['ema15'].iloc[-1]
        
        signal = new_df.iloc[-1,-1]
        pre_sig = new_df.iloc[-2,-1]
    
        if (signal == True) & (pre_sig == False):              
            print('buy success')
            #exchange.create_order('DOGE/USDT','market','buy',35)
        
        elif (signal == False) & (pre_sig == True):    
            print('sell success')
            #exchange.create_order('DOGE/USDT','market','sell', 35)
        
        
        print(new_df)

In [12]:
schedule.every(60).seconds.do(get_realtime)

Every 60 seconds do get_realtime() (last run: [never], next run: 2022-12-09 19:16:25)

In [13]:
while True:
    schedule.run_pending()
    time.sleep(1)

C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['ema5'].iloc[-1] = ta.ema(new_df['close'][-5:], length=5).iloc[-1]
C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['ema15'].iloc[-1] = ta.ema(new_df['close'][-15:], length=15).iloc[-1]
C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

             timestamp      open      high       low     close     volume  \
0  2022-12-09 11:52:00  17249.32  17249.42  17237.69  17244.78  231.97986   
1  2022-12-09 11:53:00  17245.12  17255.01  17240.90  17250.27  267.89714   
2  2022-12-09 11:54:00  17250.35  17256.50  17249.22  17255.22  176.65777   
3  2022-12-09 11:55:00  17255.22  17255.86  17250.12  17253.91  198.23726   
4  2022-12-09 11:56:00  17253.91  17255.99  17249.51  17252.19  188.19322   
5  2022-12-09 11:57:00  17251.75  17252.68  17245.46  17248.20  197.80536   
6  2022-12-09 11:58:00  17248.20  17252.47  17246.96  17248.84  134.28152   
7  2022-12-09 11:59:00  17248.84  17248.84  17239.56  17241.44  214.51041   
8  2022-12-09 12:00:00  17241.16  17246.42  17241.16  17243.58  160.52381   
9  2022-12-09 12:01:00  17243.76  17247.98  17243.55  17245.67  140.72295   
10 2022-12-09 12:02:00  17246.00  17249.52  17244.24  17245.14  111.45900   
11 2022-12-09 12:03:00  17245.56  17248.68  17244.25  17246.35  121.23380   

C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['ema5'].iloc[-1] = ta.ema(new_df['close'][-5:], length=5).iloc[-1]
C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['ema15'].iloc[-1] = ta.ema(new_df['close'][-15:], length=15).iloc[-1]
C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

             timestamp      open      high       low     close     volume  \
0  2022-12-09 11:53:00  17245.12  17255.01  17240.90  17250.27  267.89714   
1  2022-12-09 11:54:00  17250.35  17256.50  17249.22  17255.22  176.65777   
2  2022-12-09 11:55:00  17255.22  17255.86  17250.12  17253.91  198.23726   
3  2022-12-09 11:56:00  17253.91  17255.99  17249.51  17252.19  188.19322   
4  2022-12-09 11:57:00  17251.75  17252.68  17245.46  17248.20  197.80536   
5  2022-12-09 11:58:00  17248.20  17252.47  17246.96  17248.84  134.28152   
6  2022-12-09 11:59:00  17248.84  17248.84  17239.56  17241.44  214.51041   
7  2022-12-09 12:00:00  17241.16  17246.42  17241.16  17243.58  160.52381   
8  2022-12-09 12:01:00  17243.76  17247.98  17243.55  17245.67  140.72295   
9  2022-12-09 12:02:00  17246.00  17249.52  17244.24  17245.14  111.45900   
10 2022-12-09 12:03:00  17245.56  17248.68  17244.25  17246.35  121.23380   
11 2022-12-09 12:04:00  17246.42  17252.63  17245.53  17252.13  192.98023   

C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['ema5'].iloc[-1] = ta.ema(new_df['close'][-5:], length=5).iloc[-1]
C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['ema15'].iloc[-1] = ta.ema(new_df['close'][-15:], length=15).iloc[-1]
C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

...

C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['ema5'].iloc[-1] = ta.ema(new_df['close'][-5:], length=5).iloc[-1]
C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['ema15'].iloc[-1] = ta.ema(new_df['close'][-15:], length=15).iloc[-1]
C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

             timestamp      open      high       low     close      volume  \
0  2022-12-09 12:38:00  17257.00  17257.31  17252.29  17254.73   105.39565   
1  2022-12-09 12:39:00  17254.73  17265.56  17253.74  17263.86   284.04730   
2  2022-12-09 12:40:00  17263.86  17282.23  17260.85  17264.75   631.32716   
3  2022-12-09 12:41:00  17264.75  17360.00  17263.24  17283.02  3470.98860   
4  2022-12-09 12:42:00  17282.31  17297.56  17232.80  17242.81  1189.03953   
5  2022-12-09 12:43:00  17242.81  17252.05  17210.85  17220.37   934.68488   
6  2022-12-09 12:44:00  17219.83  17232.69  17204.07  17223.34   819.72363   
7  2022-12-09 12:45:00  17223.34  17229.75  17202.00  17204.56   819.52177   
8  2022-12-09 12:46:00  17204.55  17218.05  17170.85  17217.85  1425.74823   
9  2022-12-09 12:47:00  17217.85  17218.24  17196.01  17202.36   549.66270   
10 2022-12-09 12:48:00  17202.36  17214.76  17200.13  17209.81   406.93930   
11 2022-12-09 12:49:00  17209.81  17223.22  17208.22  17220.24  

C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['ema5'].iloc[-1] = ta.ema(new_df['close'][-5:], length=5).iloc[-1]
C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['ema15'].iloc[-1] = ta.ema(new_df['close'][-15:], length=15).iloc[-1]
C:\Users\Yanak\AppData\Local\Temp/ipykernel_10016/94813303.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

             timestamp      open      high       low     close      volume  \
0  2022-12-09 12:39:00  17254.73  17265.56  17253.74  17263.86   284.04730   
1  2022-12-09 12:40:00  17263.86  17282.23  17260.85  17264.75   631.32716   
2  2022-12-09 12:41:00  17264.75  17360.00  17263.24  17283.02  3470.98860   
3  2022-12-09 12:42:00  17282.31  17297.56  17232.80  17242.81  1189.03953   
4  2022-12-09 12:43:00  17242.81  17252.05  17210.85  17220.37   934.68488   
5  2022-12-09 12:44:00  17219.83  17232.69  17204.07  17223.34   819.72363   
6  2022-12-09 12:45:00  17223.34  17229.75  17202.00  17204.56   819.52177   
7  2022-12-09 12:46:00  17204.55  17218.05  17170.85  17217.85  1425.74823   
8  2022-12-09 12:47:00  17217.85  17218.24  17196.01  17202.36   549.66270   
9  2022-12-09 12:48:00  17202.36  17214.76  17200.13  17209.81   406.93930   
10 2022-12-09 12:49:00  17209.81  17223.22  17208.22  17220.24   459.93603   
11 2022-12-09 12:50:00  17221.13  17237.75  17217.61  17236.81  

KeyboardInterrupt: 